# Stochastic Gradient Descent

Stochastic Gradient Descent (SGD) is a simple yet very efficient optimization algorithm used to find the values of parameters (coefficients) of a function that minimizes a cost function. In this notebook, we will implement the SGD algorithm from scratch and apply it to learn custom word embeddings for a mock dataset. This implementation will demonstrate how words can be represented as vectors in a continuous space, capturing semantic relationships between them.

We begin by initializing the embeddings for each item in the dataset. Each word is represented as a 2-dimensional vector with randomly initialized values between -1 and 1. These vectors will be updated during training to capture the semantic relationships between words. Examine the mock dataset of 5 words and their corresponding randomly initialized embeddings below.



In [16]:
# Initialize embeddings - each word is represented as a 2D vector
# The values are randomly initialized between -1 and 1

import numpy as np
import random
import math
import sympy as sp

# Initialize embeddings
embeddings = {
    "the": [0.1, -0.2],
    "cat": [-0.3, 0.4],
    "sat": [0.5, 0.1],
    "on": [-0.1, 0.3],
    "mat": [0.2, -0.4]
}


**Activation Function: Sigmoid**

The sigmoid function is a crucial component in our implementation that serves several purposes:

1. **Range Compression**: It maps any real number into a value between 0 and 1
   σ(x) = 1 / (1 + e^(-x))

2. **Probability Interpretation**: In our context, we use sigmoid to convert dot products of word vectors into probabilities. This helps us model the likelihood of words occurring together.

3. **Properties**:
   - Output is always between 0 and 1
   - Differentiable, which is essential for gradient descent
   - S-shaped curve that provides non-linear transformation
   - Symmetric around 0.5

In [17]:
# Sigmoid function - used to convert dot products into probabilities
# The function maps any real number to a value between 0 and 1
def sigmoid(x):

      return 1 / (1 + np.exp(-x))

**Loss Function**

Our implementation uses a specialized loss function designed for word embeddings that combines:

1. **Positive Sampling Loss**: For words that appear together in context
   -log(σ(v_t · v_c))
   where:
   - v_t is the target word vector
   - v_c is the context word vector
   - · denotes dot product
   - σ is the sigmoid function

2. **Negative Sampling Loss**: For randomly selected negative examples
   -log(σ(-v_t · v_n))
   where v_n is the negative sample word vector

The complete loss function is:
L = -log(σ(v_t · v_c)) - log(σ(-v_t · v_n))

This loss function:
- Maximizes the probability of word pairs that occur together in the text
- Minimizes the probability of random word pairs (negative samples)
- Is differentiable, allowing us to use gradient descent for optimization
- Helps learn word embeddings that capture semantic relationships

The negative sampling approach is computationally efficient compared to calculating probabilities over the entire vocabulary, making it practical for large-scale word embedding training.


Let us now find the derivative of the loss function using symbolic mathematics (sympy). The loss function has three components: the target word embedding (v_t), the context word embedding (v_c), and the negative sample embedding (v_n). We compute the partial derivatives with respect to each of these components to determine how to update the embeddings during training.



In [18]:
v_t = sp.symbols('v_t')
v_c = sp.symbols('v_c')
v_n = sp.symbols('v_n')

f = -sp.log(1 / (1 + sp.exp(v_t*v_c))) - sp.log(1 / (1 + sp.exp(-v_t*v_n)))

print("Derivative w.r.t v_t : ", sp.diff(f, v_t))
print("Derivative w.r.t v_c : ", sp.diff(f, v_c))
print("Derivative w.r.t v_n : ", sp.diff(f, v_n))

Derivative w.r.t v_t :  v_c*exp(v_c*v_t)/(exp(v_c*v_t) + 1) - v_n*exp(-v_n*v_t)/(1 + exp(-v_n*v_t))
Derivative w.r.t v_c :  v_t*exp(v_c*v_t)/(exp(v_c*v_t) + 1)
Derivative w.r.t v_n :  -v_t*exp(-v_n*v_t)/(1 + exp(-v_n*v_t))


In [19]:
def update_embeddings(target, context, negative, alpha=0.01):
    """
    Updates word embeddings using stochastic gradient descent.

    The function implements the following steps:
    1. Computes dot products between target-context and target-negative pairs
    2. Calculates loss using the formula: -log(σ(v_t · v_c)) - log(σ(-v_t · v_n))
    3. Computes gradients for all three vectors
    4. Updates the embeddings using the computed gradients

    Parameters:
    - target: The target word whose embedding is being updated
    - context: The context word that appears near the target
    - negative: A randomly sampled word used as a negative example
    - alpha: Learning rate for gradient descent

    Returns:
    - loss: The current value of the loss function
    """

    v_target = np.array(embeddings[target])
    v_context = np.array(embeddings[context])
    v_negative = np.array(embeddings[negative])

    dot_product_target_context = np.dot(v_target, v_context)
    dot_product_target_negative = np.dot(v_target, v_negative)

    loss = -np.log(sigmoid(dot_product_target_context)) - np.log(sigmoid(-dot_product_target_negative))
    
    # FIX: Define sigma_pos and sigma_neg before they are used in the gradients
    sigma_pos = sigmoid(dot_product_target_context)
    sigma_neg = sigmoid(-dot_product_target_negative)

    grad_target = v_context * (sigma_pos - 1) + v_negative * (sigma_neg - 1)
    grad_context = v_target * (sigma_pos - 1)
    grad_negative = v_target * (1 - sigma_neg)

    # TODO: Update embeddings

    embeddings[target] = v_target - alpha * grad_target
    embeddings[context] = v_context - alpha * grad_context
    embeddings[negative] = v_negative - alpha * grad_negative

    return loss

Now we'll train our word embeddings using SGD. We define word pairs based on words that appear together in our mock dataset. For simplicity, we use a fixed negative sample, though in practice this would be randomly sampled. The training process runs for 100 epochs, with the word pairs shuffled in each epoch to improve convergence. We print the loss every 10 epochs to monitor the training progress.



In [20]:
# Word pairs and negative sampling
word_pairs = [("cat", "the"), ("sat", "cat"), ("on", "sat"), ("mat", "on"), ("the", "mat")]
negative_sample = "on"

# Run SGD for 100 epochs
for epoch in range(100):
    random.shuffle(word_pairs)  # Shuffle word pairs
    total_loss = 0
    for target, context in word_pairs:
        loss = update_embeddings(target, context, negative_sample)
        total_loss += loss
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {total_loss:.4f}")

print("Final embeddings:", embeddings)

Epoch 0, Loss: 7.0981
Epoch 10, Loss: 7.0751
Epoch 20, Loss: 7.0533
Epoch 30, Loss: 7.0317
Epoch 40, Loss: 7.0090
Epoch 50, Loss: 6.9848
Epoch 60, Loss: 6.9602
Epoch 70, Loss: 6.9336
Epoch 80, Loss: 6.9066
Epoch 90, Loss: 6.8788
Final embeddings: {'the': array([-0.02048059, -0.02296681]), 'cat': array([-0.18087267,  0.58649518]), 'sat': array([0.23673144, 0.5052219 ]), 'on': array([-0.17716375,  0.0030742 ]), 'mat': array([ 0.06394589, -0.28126674])}


If you are wondering what these embeddings are, they are a way to represent words in a vector space. These embeddings are learned by the model during training and are used to represent words in a way that captures their meaning and relationships with other words. For example, words that are similar in meaning will have embeddings that are close to each other in the vector space.

As a demonstration, let us use pre-trained embeddings from the Word2Vec model to find the embeddings for the words in our mock dataset. We will then compare these embeddings with the embeddings learned by our model. The Word2Vec model is trained on a far bigger dataset and is expected to have better embeddings than the ones learned by our model.

To visualize these embeddings, you could:
1. Plot them as points in a 2D space since they are 2-dimensional vectors
2. Calculate and compare the cosine similarities between different word pairs
3. Use dimensionality reduction techniques like t-SNE or PCA if working with higher-dimensional embeddings